In [1]:
import tensorflow as tf
import keras
from keras import layers

In [2]:
def load_data_artifact(dir_path):
    train, valid = tf.keras.preprocessing.image_dataset_from_directory(
        dir_path,
        label_mode='binary',
        image_size=[200, 200],
        validation_split=0.2,
        subset='both',
        shuffle=True,
        seed=42,
        interpolation='area',
        batch_size=256
    )

    return train, valid

In [3]:
def create_model():
    backbone = keras.applications.VGG19(False, 'imagenet', input_shape=(200, 200, 3))

    for layer in backbone.layers:
        layer.trainable = False

    flatten = layers.Flatten()(backbone.output)
    dense = layers.Dense(128, activation='elu')(flatten)
    norm = layers.BatchNormalization()(dense)

    dense = layers.Dense(128, activation='elu')(norm)
    norm = layers.BatchNormalization()(dense)

    dense = layers.Dense(1, activation='sigmoid')(norm)

    adam = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model = tf.keras.Model(inputs=backbone.input, outputs=dense)
    model.compile(
        optimizer=adam,
        loss='binary_crossentropy',
        metrics=['accuracy', 'mse', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    model.summary()
    return model

In [4]:
def train_model(model: tf.keras.Model, train_data, validation_data, model_name):
    # train_data = train_data.map(tf.keras.applications.vgg19.preprocess_input)
    # validation_data = validation_data.map(tf.keras.applications.vgg19.preprocess_input)

    saving_models = keras.callbacks.ModelCheckpoint("./saved_models/checkpoints/"+model_name+".{epoch:02d}.h5")
    tensorboard = keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)
    return model.fit(train_data, validation_data=validation_data, epochs=7, batch_size=256, callbacks=[saving_models, tensorboard])

In [5]:
model = create_model()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [6]:
train, valid = load_data_artifact("../../datasets/artifact")

Found 1934329 files belonging to 2 classes.
Using 1547464 files for training.
Using 386865 files for validation.


In [7]:
train_small = train.take(32).map(lambda x, y: (tf.keras.applications.vgg19.preprocess_input(x), y))
valid_small = valid.take(16).map(lambda x, y: (tf.keras.applications.vgg19.preprocess_input(x), y))

In [8]:
history = train_model(model, train_small, valid_small, "artifact-2-2")

Epoch 1/7
32/32 [==============================] - 106s 3s/step - loss: 0.7973 - accuracy: 0.5493 - mse: 0.2827 - precision: 0.5298 - recall: 0.5269 - val_loss: 0.8537 - val_accuracy: 0.5933 - val_mse: 0.2771 - val_precision: 0.5576 - val_recall: 0.7217
Epoch 2/7
32/32 [==============================] - 104s 3s/step - loss: 0.6646 - accuracy: 0.6464 - mse: 0.2296 - precision: 0.6483 - recall: 0.5984 - val_loss: 0.7621 - val_accuracy: 0.6223 - val_mse: 0.2534 - val_precision: 0.5825 - val_recall: 0.7411
Epoch 3/7
32/32 [==============================] - 105s 3s/step - loss: 0.6111 - accuracy: 0.6797 - mse: 0.2091 - precision: 0.6747 - recall: 0.6328 - val_loss: 0.7208 - val_accuracy: 0.6362 - val_mse: 0.2421 - val_precision: 0.5961 - val_recall: 0.7416
Epoch 4/7
32/32 [==============================] - 102s 3s/step - loss: 0.5693 - accuracy: 0.7035 - mse: 0.1931 - precision: 0.7099 - recall: 0.6541 - val_loss: 0.6906 - val_accuracy: 0.6445 - val_mse: 0.2332 - val_precision: 0.6073 - val